In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import os.path as op
import shutil
from zipfile import ZipFile
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from utils import get_image_paths, word_from_image_path, preprocess_image, print_im, TextTransform, N_CHARS, SEQUENCE_LENGTH, IMAGE_DIMENSIONS
from utils import NgramTransform, load_trained_CNN_weights
from multi_gpu import make_parallel

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


#### Building N-grams Neural Network

In [3]:
cnn = load_trained_CNN_weights('p_model_30.h5')

In [4]:
cnn.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convo0 (Convolution2D)           (None, 32, 100, 64)   1664        convolution2d_input_2[0][0]      
____________________________________________________________________________________________________
maxpooling2d_6 (MaxPooling2D)    (None, 16, 50, 64)    0           convo0[0][0]                     
____________________________________________________________________________________________________
convo1 (Convolution2D)           (None, 16, 50, 5)     5242885     maxpooling2d_6[0][0]             
____________________________________________________________________________________________________
maxpooling2d_7 (MaxPooling2D)    (None, 8, 25, 5)      0           convo1[0][0]                     
___________________________________________________________________________________________

### Loading Images and Batch

In [3]:
from tqdm import tqdm, tqdm_notebook
base_dir = '/mnt/c/Users/benoua/Google Drive/Master X - Data Sciences/2_Structured Data/Structured-Data/mjsynth'
# base_dir = '/mnt/mjsynth/mnt/ramdisk/max/90kDICT32px/'

images_paths = get_image_paths(base_dir)
#%%time
ims = []
errors_1, errors_2 = [], []
for i, im in enumerate(images_paths[:12]):
    try:
        ims.append(preprocess_image(img_to_array(load_img(im, grayscale=True))))
    except OSError:
        errors_1.append(i)

In [4]:
batch = np.stack(ims).reshape(12, 32, 100, 1)
batch.shape

(12, 32, 100, 1)

In [5]:
### NGram model output vector 
ngram_transformer = NgramTransform()

In [6]:
batch_y = ngram_transformer.make_batch_labels(images_paths[:12], False)
batch_y = batch_y.reshape(batch_y.shape[0], batch_y.shape[2]).astype(float)

### Predictions

In [7]:
predictions = cnn.predict(batch)

In [8]:
import pickle
with open('tt_new.pickle', 'rb') as f:
    tt = pickle.load(f)

In [9]:
for i in range(12): 
    print ("{} \t\t {}".format(tt.word_from_matrix(predictions[i]), word_from_image_path(images_paths[i])))

weolhn                  		 FOURTEENS
postetesssss            		 RECKONINGS
viess                   		 requisite
oyees                   		 RELIGHTS
sonseses                		 STIPULATES
oyees                   		 Acidified
voxes                   		 DUMB
wiles                   		 SUITING
wises                   		 altruists
weodhnss                		 pillage
weels                   		 SHEBANG
vokss                   		 nantes
